In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelBinarizer, LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tens
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime




In [3]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
##Encode Catagorical Veriables
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])



In [5]:
##One Hot Encode Geography column
one_hot_encoder_geography = OneHotEncoder()
geo_encode_value = one_hot_encoder_geography.fit_transform(data[['Geography']])

In [6]:
one_hot_encoder_geography.get_feature_names_out(['Geography'])
geo_encoder_pd=pd.DataFrame(geo_encode_value.toarray(),columns=one_hot_encoder_geography.get_feature_names_out(['Geography']))

In [7]:
## combine all the onde hot encoded columns with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoder_pd],axis=1)

In [8]:
## save the encpder and scelar
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_geography.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geography,file)


In [9]:
## Divide the dataset into independent and dependent features
data.head()
X= data.drop('Exited', axis=1)
y=data['Exited']

## split the data in training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
## scale these features 
scaler=StandardScaler()
""" print(X_train.dtypes)
print(X_train.head())
for col in X_train.columns:
    # This will show rows that can't be converted to float in this column
    bad_rows = pd.to_numeric(X_train[col], errors='coerce').isna()
    if bad_rows.any():
        print(f"Problem in column: {col}")
        print(X_train[bad_rows][col]) """

X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [10]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [11]:
## Build ANN Model
## Step 1: Sequential model 64 in dense is the number of nodes, 64 is assumption we can take any number and 
##then we define the inout shape like how many elements are comming to the dense layer(nurones (nodes) 64 number) 
##the elements will be connected to each nuron in Dense layer 
## 1st hideen layer connected with input layer: (X_train.shape[1],))--> this is single diamention and have 12 inputs
model=Sequential([
    Dense(64,activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'), ## hidden layer 2
    Dense(1,activation='sigmoid') ## output layer
])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [13]:
model.compile(optimizer=opt,loss="binary_crossentropy", metrics=['accuracy'])


In [14]:
##Setup the Tensorboard 
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [15]:
## Setup early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [16]:
history=model.fit(X_train, y_train, validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8183 - loss: 0.4425 - val_accuracy: 0.8575 - val_loss: 0.3453
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step - accuracy: 0.8596 - loss: 0.3489 - val_accuracy: 0.8590 - val_loss: 0.3463
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step - accuracy: 0.8510 - loss: 0.3505 - val_accuracy: 0.8575 - val_loss: 0.3508
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step - accuracy: 0.8560 - loss: 0.3467 - val_accuracy: 0.8565 - val_loss: 0.3434
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step - accuracy: 0.8577 - loss: 0.3370 - val_accuracy: 0.8555 - val_loss: 0.3579
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step - accuracy: 0.8671 - loss: 0.3291 - val_accuracy: 0.8625 - val_loss: 0.3454
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step - accuracy: 0.8687 - loss: 0.3279 - val_accuracy: 0.8605 - val_loss: 0.3404
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - accuracy: 0.8635 - loss: 0.3

In [17]:
model.save('model.h5')
##Load Tensorboard Extention
%load_ext tensorboard
%tensorboard --logdir logs/fit